In [197]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

In [200]:
file_list = []
for dirname, _, filenames in os.walk(f"{AMZN_folder}"):
    for filename in filenames:
        file_list.append(os.path.join(dirname,filename))leasesleasecostdetails,terseLabel,period_type,label,2020-03-31,2019-12-31


SyntaxError: invalid syntax (<ipython-input-200-97fb4e07460e>, line 4)

In [201]:
list_num = 2
print(file_list[list_num])
df = pd.read_csv(file_list[list_num],index_col=[0,1])
df

../data/timeseries/AMZN/1002000 - Statement - Consolidated Statements of Operations.csv


,,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,...,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31,2008-09-30,2008-06-30,2007-12-31
label,xbrl_tag,,,,,,,,,,,,,,,,,,,,,
Total net sales,us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,7.545200e+10,2.805220e+11,6.998100e+10,6.340400e+10,5.970000e+10,2.328870e+11,5.657600e+10,5.288600e+10,5.104200e+10,1.778660e+11,...,6.566000e+09,7.131000e+09,2.450900e+10,5.449000e+09,4.651000e+09,4.889000e+09,1.916600e+10,4.264000e+09,4.063000e+09,1.483500e+10
Product [Member],us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,4.184100e+10,1.604080e+11,3.972600e+10,3.585600e+10,3.428300e+10,1.419150e+11,3.374600e+10,3.186400e+10,3.160500e+10,1.185730e+11,...,NaN,NaN,2.227300e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Service [Member],us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,3.361100e+10,1.201140e+11,3.025500e+10,2.754800e+10,2.541700e+10,9.097200e+10,2.283000e+10,2.102200e+10,1.943700e+10,5.929300e+10,...,NaN,NaN,2.236000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Goods and Services Sold,us-gaap_costofgoodsandservicessold,4.425700e+10,1.655360e+11,4.130200e+10,3.633700e+10,3.392000e+10,1.391560e+11,3.300300e+10,3.063200e+10,3.073500e+10,1.119340e+11,...,4.957000e+09,5.501000e+09,1.897800e+10,4.176000e+09,3.518000e+09,3.741000e+09,1.489600e+10,3.265000e+09,3.096000e+09,1.148200e+10
Fulfillment Expense,amzn_fulfillmentexpense,1.153100e+10,4.023200e+10,1.016700e+10,9.271000e+09,8.601000e+09,3.402700e+10,8.275000e+09,7.932000e+09,7.792000e+09,2.524900e+10,...,5.820000e+08,5.460000e+08,2.052000e+09,4.660000e+08,4.090000e+08,4.220000e+08,1.658000e+09,3.930000e+08,3.610000e+08,1.292000e+09
Marketing Expense,us-gaap_marketingexpense,4.828000e+09,1.887800e+10,4.752000e+09,4.291000e+09,3.664000e+09,1.381400e+10,3.303000e+09,2.901000e+09,2.699000e+09,1.006900e+10,...,2.110000e+08,2.010000e+08,6.800000e+08,1.490000e+08,1.290000e+08,1.280000e+08,4.820000e+08,1.080000e+08,1.020000e+08,3.440000e+08
Technology And Content Expense,amzn_technologyandcontentexpense,9.325000e+09,3.593100e+10,9.200000e+09,9.065000e+09,7.927000e+09,2.883700e+10,7.162000e+09,7.247000e+09,6.759000e+09,2.262000e+10,...,4.080000e+08,3.660000e+08,1.240000e+09,3.150000e+08,2.990000e+08,2.750000e+08,1.033000e+09,2.640000e+08,2.580000e+08,8.180000e+08
General and Administrative Expense,us-gaap_generalandadministrativeexpense,1.452000e+09,5.203000e+09,1.348000e+09,1.270000e+09,1.173000e+09,4.336000e+09,1.041000e+09,1.111000e+09,1.067000e+09,3.674000e+09,...,1.130000e+08,9.700000e+07,3.280000e+08,8.300000e+07,7.700000e+07,6.800000e+07,2.790000e+08,7.300000e+07,7.400000e+07,2.350000e+08
"Other Operating Income (Expense), Net",us-gaap_otheroperatingincomeexpensenet,-7.000000e+07,NaN,NaN,NaN,5.000000e+06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
df.columns

Index(['2020-03-31', '2019-12-31', '2019-09-30', '2019-06-30', '2019-03-31',
       '2018-12-31', '2018-09-30', '2018-06-30', '2018-03-31', '2017-12-31',
       '2017-09-30', '2017-06-30', '2017-03-31', '2016-12-31', '2016-09-30',
       '2016-06-30', '2016-03-31', '2015-12-31', '2015-09-30', '2015-06-30',
       '2015-03-31', '2014-12-31', '2014-09-30', '2014-06-30', '2014-03-31',
       '2013-12-31', '2013-09-30', '2013-06-30', '2013-03-31', '2012-12-31',
       '2012-09-30', '2012-06-30', '2012-03-31', '2011-12-31', '2011-09-30',
       '2011-06-30', '2011-03-31', '2010-12-31', '2010-09-30', '2010-06-30',
       '2010-03-31', '2009-12-31', '2009-09-30', '2009-06-30', '2009-03-31',
       '2008-12-31', '2008-09-30', '2008-06-30', '2007-12-31'],
      dtype='object')

In [165]:
280522000000.0-6.998100e+10-6.340400e+10-5.970000e+10

87437000000.0

In [87]:
(datetime.strptime(df.columns.max(), "%Y-%m-%d")-datetime.strptime(df.columns.min(), "%Y-%m-%d")).days/(365/4)



43.035616438356165